**Aisha Muhammad Nawaz  L200921**  
PySpark Class Activity
8A BSCS MMD  
23rd April 2024

Instructions:
2. Write efficient Spark code for creating K-shingles given a huge document and K as input.
3. Write an efficient SPARK code for Minhashing (uses the logic of hash functions as shown in the uploaded slide). The map reduce code is given in slides.


In [1]:
# #Running on Colab
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-8-openjdk-amd64'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=46fb6e6cffc7c6d40c7dc511d904e7bcd1a55447f58431d4859af001a126bf37
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark
The following additional packages will be installed:
  libxtst6 openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra fonts-nanum fonts-ipafont-gothic
  fonts-ipafont-mincho fonts-wqy-microhei fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  libxtst6 openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 3 newly installed, 0 to remove and 45 not upgraded.
Need to get 39.7 MB of archives.
After this operation, 144 MB of additional disk space will be used.
Selecting previously unselected package

In [2]:
!sudo apt update

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [814 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:8 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,748 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://security.ubuntu.com/ubunt

In [3]:
#  Import the libraries we will need
import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

# Create Spark session and ContextRun PySpark.
# create the session
conf = SparkConf().set("spark.ui.port","4050")
# create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.appName("DataFrame").config('spark.ui.port', '4050').getOrCreate()
spark

In [83]:
# Q2: Write efficient Spark code for creating K-shingles given a huge document and K as input.
import hashlib
HugeDocument = sc.parallelize(['D1,Pretend it is docu','D2,Pretend it no docu'])
K=9
d=0

def hashIt(shingle):
    # Hash the shingle to 4 bytes
    buckets = 8
    hashObj = hashlib.sha256(shingle.encode())
    hashed = int.from_bytes(hashObj.digest(), byteorder='big') % buckets
    return hashed
def getShingles(line):
    global K
    documentNumber, text = line.split(',')
    text = text.lower()
    setOfShingles = set()
    for i in range(len(text) - K + 1):
        shingle = text[i:i+K]
        hashedShingle = hashIt(shingle)
        setOfShingles.add(hashedShingle)
    return documentNumber, setOfShingles

shingles=HugeDocument.map(lambda x: getShingles(x))

# Removing Duplicate Shingles
uniqueShingles = shingles.flatMap(lambda x: x[1]).distinct().collect()

# Getting Boolean Matrix ---->

shingleIndex = {shingle: i for i, shingle in enumerate(uniqueShingles)} # Dictionary to map each shingle to an index
docShin = shingles.map(lambda x: (x[0], [shingleIndex[shingle] for shingle in x[1]])) # List of document and shingle pairs
sparseM = docShin.flatMapValues(lambda x: x).map(lambda x: (x, 1)).reduceByKey(lambda x, y: x).sortByKey()
dfData = sparseM.map(lambda x: (x[0][0], x[0][1], x[1])).toDF(["Document", "Shingle", "Value"]) # Convert sparseM RDD to DataFrame
pivotedDf = dfData.groupby("Document").pivot("Shingle").agg({"Value": "max"})
pivotedDf = pivotedDf.fillna(0) # Null values filled with 0
pivotedDf.show()

+--------+---+---+---+---+---+---+---+---+
|Document|  0|  1|  2|  3|  4|  5|  6|  7|
+--------+---+---+---+---+---+---+---+---+
|      D1|  1|  1|  1|  0|  1|  1|  1|  1|
|      D2|  1|  0|  1|  1|  1|  1|  1|  0|
+--------+---+---+---+---+---+---+---+---+



In [84]:
# Q3:  Write an efficient SPARK code for Minhashing
import random

# 100 random permutations of the rows
KPermutations = [random.sample(range(len(uniqueShingles)), len(uniqueShingles)) for _ in range(100)]

def updateSignature(row, permutations):
    document, shingleIndices = row
    updatedSig = [float('inf')] * len(permutations)
    for shingleIndex in shingleIndices:
        for i, perm in enumerate(permutations):
            if perm.index(shingleIndex) < updatedSig[i]:
                updatedSig[i] = perm.index(shingleIndex)
    return document, updatedSig

# Update signature matrix with min-hash
signatureMatrix = docShin.map(lambda x: updateSignature(x, KPermutations))
signatureDF = signatureMatrix.toDF(["Document", "Signature"])
signatureDF.show(truncate=False)

+--------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|Document|Signature                                                                                                                                                                                                                                                                                                   |
+--------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|D1      |[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0,